In [2]:
import requests
from dotenv import load_dotenv
import os

# Replace with the full path to your .env file
load_dotenv(dotenv_path="/Users/ioa6870/repos/tbcrozier/diet-coach/.env")

api_key = os.getenv("USDA_API_KEY")
print("Loaded API key:", api_key)

def get_basic_nutrients(food_name, api_key):
    url = "https://api.nal.usda.gov/fdc/v1/foods/search"
    params = {
        "query": food_name,
        "pageSize": 1,
        "api_key": api_key
    }
    response = requests.get(url, params=params)
    data = response.json()

    # Get the first search result
    if not data.get("foods"):
        return f"No foods found for '{food_name}'"

    food = data["foods"][0]
    nutrients = {
        "calories": None,
        "protein_g": None,
        "carbs_g": None,
        "fat_g": None
    }

    for nutrient in food.get("foodNutrients", []):
        if nutrient["nutrientId"] == 1008:
            nutrients["calories"] = nutrient["value"]
        elif nutrient["nutrientId"] == 1003:
            nutrients["protein_g"] = nutrient["value"]
        elif nutrient["nutrientId"] == 1004:
            nutrients["fat_g"] = nutrient["value"]
        elif nutrient["nutrientId"] == 1005:
            nutrients["carbs_g"] = nutrient["value"]

    return {
        "description": food["description"],
        "fdcId": food["fdcId"],
        **nutrients
    }

# Example use:
print(get_basic_nutrients("potato", api_key))


Loaded API key: xhAZ31hvj9ubCwpHsSNXQxtXu1QVG6G4svvOh6eT
{'description': 'Potato patty', 'fdcId': 2709510, 'calories': 171, 'protein_g': 3.89, 'carbs_g': 13.49, 'fat_g': 11.32}


In [3]:
def estimate_meal_nutrition(meal_description, api_key):
    food_items = [item.strip() for item in meal_description.split(",")]

    total = {"calories": 0, "protein_g": 0, "carbs_g": 0, "fat_g": 0}
    details = []

    for food in food_items:
        result = get_basic_nutrients(food, api_key)
        if isinstance(result, dict):
            details.append(result)
            for key in total:
                total[key] += result.get(key, 0) or 0

    return {
        "meal_description": meal_description,
        "total_nutrition": total,
        "items": details
    }

meal = "3 eggs, toast with butter, a banana, coffee with cream"
print(estimate_meal_nutrition(meal, api_key))


{'meal_description': '3 eggs, toast with butter, a banana, coffee with cream', 'total_nutrition': {'calories': 986.0, 'protein_g': 29.65, 'carbs_g': 170.92, 'fat_g': 24.11}, 'items': [{'description': 'Eggs, Grade A, Large, egg white', 'fdcId': 747997, 'calories': 55.0, 'protein_g': 10.7, 'carbs_g': 2.36, 'fat_g': 0.0}, {'description': 'Melba toast', 'fdcId': 2707702, 'calories': 390, 'protein_g': 12.1, 'carbs_g': 76.6, 'fat_g': 3.2}, {'description': 'Bananas, dehydrated, or banana powder', 'fdcId': 173945, 'calories': 346, 'protein_g': 3.89, 'carbs_g': 88.3, 'fat_g': 1.81}, {'description': 'Cream, fluid, light (coffee cream or table cream)', 'fdcId': 170857, 'calories': 195, 'protein_g': 2.96, 'carbs_g': 3.66, 'fat_g': 19.1}]}


In [ ]:
import subprocess

def estimate_nutrition_with_ollama(meal_description):
    prompt = f"""Estimate the total calories, protein (g), carbs (g), and fat (g) in this meal:

    "{meal_description}"

    Return a JSON object like this:
    {{
        "calories": ...,
        "protein_g": ...,
        "carbs_g": ...,
        "fat_g": ...
    }}
    """

    result = subprocess.run(
        ["ollama", "run", "llama2"],
        input=prompt.encode(),
        capture_output=True
    )

    return result.stdout.decode().strip()

# meal = "3 scrambled eggs with cheese, 2 slices of toast, and a cup of black coffee"
# print(estimate_nutrition_with_ollama(meal))

In [6]:
meal = "3 scrambled eggs with cheese, 2 slices of toast, and a cup of black coffee"
print(estimate_nutrition_with_ollama(meal))


Sure! To estimate the nutritional content of this meal, we can use a nutrition calculator or consult a reliable nutrition source. Here are the approximate values for each component:

* 3 scrambled eggs:
	+ Calories: approximately 140-160 calories (depending on size and cooking method)
	+ Protein: approximately 12-14 grams (depending on size and egg quality)
	+ Carbs: approximately 0 grams (since eggs are low in carbohydrates)
	+ Fat: approximately 0 grams (since eggs are very low in fat)
* 2 slices of toast:
	+ Calories: approximately 150-200 calories (depending on slice size and bread type)
	+ Protein: approximately 2-3 grams (depending on bread type and toasting method)
	+ Carbs: approximately 20-25 grams (depending on slice size and bread type)
	+ Fat: approximately 1-2 grams (depending on bread type and toasting method)
* 1 cup of black coffee:
	+ Calories: approximately 0 calories (since coffee has no caloric value)
	+ Protein: approximately 0 grams (since coffee has no protein co

# Ollama + BQ Insert

In [51]:
import subprocess
import json
import re

def estimate_nutrition_with_ollama(meal_description):
    prompt = f"""
    Estimate the total calories, protein (g), carbs (g), and fat (g) in this meal:

    "{meal_description}"

    Return ONLY a valid JSON object, no explanation, no markdown, just:
    {{
        "calories": 500,
        "protein_g": 25,
        "carbs_g": 40,
        "fat_g": 20
    }}
    """

    result = subprocess.run(
        ["ollama", "run", "llama2"],
        input=prompt.encode(),
        capture_output=True
    )

    raw_output = result.stdout.decode().strip()

    # Clean up markdown/code block formatting if present
    cleaned_output = re.sub(r"```(json)?", "", raw_output).strip()

    try:
        parsed = json.loads(cleaned_output)
        return parsed, raw_output
    except json.JSONDecodeError:
        print("❌ Could not parse LLM response — saving raw anyway.")
        return None, raw_output


#### Create BQ Table

In [52]:
from google.cloud import bigquery
from google.api_core.exceptions import NotFound

def create_meals_table_if_needed():
    client = bigquery.Client()
    table_id = "vocal-spirit-372618.diet_data.meals"  # Replace with your own project/dataset

    schema = [
        bigquery.SchemaField("timestamp", "DATETIME", mode="REQUIRED"),
        bigquery.SchemaField("user_id", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("meal_description", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("calories", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("protein_g", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("carbs_g", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("fat_g", "FLOAT", mode="NULLABLE"),
        bigquery.SchemaField("raw_response", "STRING", mode="REQUIRED"),
    ]

    try:
        client.get_table(table_id)
        print("✅ Table already exists.")
    except NotFound:
        table = bigquery.Table(table_id, schema=schema)
        client.create_table(table)
        print(f"✅ Created table {table_id}")


#### Load BQ Table

In [53]:
from datetime import datetime

def insert_meal_to_bq(user_id, meal, nutrition_data, raw_response):
    client = bigquery.Client()
    table_id = "vocal-spirit-372618.diet_data.meals"

    row = {
        "timestamp": datetime.utcnow().isoformat(),
        "user_id": user_id,
        "meal_description": meal,
        "calories": nutrition_data.get("calories"),
        "protein_g": nutrition_data.get("protein_g"),
        "carbs_g": nutrition_data.get("carbs_g"),
        "fat_g": nutrition_data.get("fat_g"),
        "raw_response": raw_response
    }

    errors = client.insert_rows_json(table_id, [row])
    if errors:
        print("❌ BigQuery insert errors:", errors)
    else:
        print("✅ Meal inserted into BigQuery.")


In [55]:
# Create table if needed (run once)
create_meals_table_if_needed()

# Log a meal
user_id = "blake@demo"
meal = "3 scrambled eggs with cheese, 2 slices of toast, and a cup of black coffee"

nutrition_data, raw_response = estimate_nutrition_with_ollama(meal)
insert_meal_to_bq(user_id, meal, nutrition_data or {}, raw_response)


✅ Table already exists.
✅ Meal inserted into BigQuery.
